<a href="https://colab.research.google.com/github/TUstudents/gpHSP/blob/master/0_prepare_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
! rm -rf gpHSP
!git clone https://github.com/aspuru-guzik-group/gpHSP

Cloning into 'gpHSP'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 147 (delta 65), reused 79 (delta 35), pack-reused 38
Receiving objects: 100% (147/147), 32.98 MiB | 30.70 MiB/s, done.
Resolving deltas: 100% (79/79), done.


# Assuming in a colab enviroment

In [3]:
import sys
if 'google.colab' in sys.modules:
    print('In colab!')
    sys.path.insert(0,'gpHSP')
    !pip install ml_collections ngboost gpflow


In colab!


In [4]:
pip install mordredcommunity[full] dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00


In [35]:
 #sys.path.insert(0,'/content/gpHSP')

In [5]:
!pwd

/content


In [6]:
sys.path

['gpHSP',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [7]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import scipy.stats as stats

import ngboost
import tensorflow as tf
import gpflow as gpf
import gphsp

gphsp.notebook_context()
gphsp.print_modules([ngboost, tf , gpf])

ngboost    = 0.5.1
tensorflow = 2.15.0
gpflow     = 2.9.1


## Utilities

In [8]:
at_data_dir = lambda x: os.path.join('gpHSP/data', x)

# Load data and get all smiles

In [9]:
files = [at_data_dir('Solvents_exp.csv'),
         at_data_dir('Polymers_exp.csv'),
         at_data_dir('sample_molecules.csv'),
         at_data_dir('HSPiP.csv')
         ]
all_smiles = []
for fname in files:
    df = pd.read_csv(fname)
    df['smiles'] = df['smiles'].apply(gphsp.get_isomeric_smiles)
    gphsp.peek_df(df)
    all_smiles.extend(df['smiles'].tolist())

all_smiles = np.unique(all_smiles)
len(all_smiles)

Index(['key', 'Type', 'δd', 'δp', 'δh', 'smiles', 'ID_type', 'ID', 'Ref',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'label',
       'finished', 'job_name', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile', 'drug'],
      dtype='object')
(193, 39)


,key,Type,δd,δp,δh,smiles,ID_type,ID,Ref,organic,...,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile,drug
0,"1,1,1-Trichloroethane",Solvent,16.8,4.3,2.0,CC(Cl)(Cl)Cl,CAS,71-55-6,1,True,...,0.126,0,16.6,-3.836,4.24,-2.021,1.54,41.428,[ 0. 0. 0. 0. ...,False


Index(['label', 'δd', 'δp', 'δh', 'smiles', 'test', 'poly_label', 'n_copies',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'n_frags',
       'largest', 'finished', 'job_name', 'poly_smiles', 'homo', 'lumo', 'gap',
       'd-moments', 'dipole', 'polar', 'run_time', 'n_cores', 'compute_time',
       'Area', 'Hba', 'Hbd', 'Volume', 'sigma_mom_0', 'sigma_mom_1',
       'sigma_mom_2', 'sigma_mom_3', 'sigma_mom_4', 'sigma_mom_5',
       'sigma_norm', 'sigma_profile'],
      dtype='object')
(31, 39)


,label,δd,δp,δh,smiles,test,poly_label,n_copies,organic,n_electrons,...,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Polyacrylonitrile-n5,20.0,15.1,7.9,CCC#N,True,Polyacrylonitrile,5,True,110,...,0.22,0.343,0,175.0,26.1,154.0,51.4,165.0,66.77,[ 0. 0. 0. 0. ...


Index(['names', 'smiles'], dtype='object')
(11, 2)


,names,smiles
0,J52-Cl,CCCCCCCC[C@@H](CCCCCC)Cn1nc2c(-c3ccc(C)s3)c(F)...


Index(['Chemical', 'CAS', 'smiles', 'HSPiP_indx', 'good_smiles', 'single_frag',
       'duplicated', 'sanity', 'label', 'organic', 'n_electrons', 'n_atoms',
       'charge', 'MolWt', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile'],
      dtype='object')
(8852, 35)


,Chemical,CAS,smiles,HSPiP_indx,good_smiles,single_frag,duplicated,sanity,label,organic,...,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Acetaldehyde,75-07-0,CC=O,0,True,True,True,True,hspip_0,True,...,0.0,0.064,0,33.0,18.4,30.6,30.1,39.1,22.203,[ 0.00000000e+00 0.00000000e+00 0.0000000...


8833

Calculate a mordred values and a mask

In [10]:
all_smiles = np.unique([gphsp.get_isomeric_smiles(s) for s in all_smiles])
all_values = gphsp.calculate_mordred(all_smiles)
mask = gphsp.calculate_mask(all_values)
np.savez_compressed(at_data_dir('mordred_features.npz'),
                    smiles = all_smiles,
                    values = all_values,
                    mask = mask)
features = gphsp.SmilesMap(at_data_dir('mordred_features.npz'))
#features(all_values)

100%|██████████| 8833/8833 [20:44<00:00,  7.10it/s]


ValueError: Cannot index with multidimensional key